# NLP Final Project

## Install package

In [ ]:
!pip install gensim
!pip install tensorflow
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
^C


## Preprocessing

In [27]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import gensim.downloader as api
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
import torch
import warnings
warnings.filterwarnings("ignore")
import nltk
import re
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, create_optimizer
from sklearn.metrics import precision_score, f1_score, classification_report
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Reshape


In [2]:
df = pd.read_csv('/content/altibbi_specialty_data.csv')[["question_body", "specialty_id"]].dropna()
df.columns = ["text", "label"]
df.head()

,text,label
0,استشاره عيون,23
1,السلام عليكم ممكن دكتور مفاصل واعصاب,14
2,عندي نقص فيتامين د هل ممكن استخدم معه كالسيوم,14
3,عمليه الحول للكبار,23
4,ألم بالكتف الايسر من فترة,14


In [3]:
nltk.download('stopwords')

stemmer = ISRIStemmer()
arabic_stopwords = set(stopwords.words("arabic"))

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[إأآا]", "ا", text)
    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ؤ", "و", text)
    text = re.sub(r"ئ", "ي", text)
    text = re.sub(r"ة", "ه", text)
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = [stemmer.stem(w) for w in text.split() if w not in arabic_stopwords and len(w) > 1]
    return " ".join(tokens)

df['text'] = df['text'].apply(preprocess_text)
print("preprocessed sentence:", df['text'].iloc[1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


preprocessed sentence: سلم علي مكن دكتور فصل عصب


In [4]:
X = df['text']
y = df['label']

In [5]:
sample_fraction = 0.3
X_sample = X.sample(frac=sample_fraction, random_state=42)
y_sample = y.loc[X_sample.index]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

In [7]:
print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 22214
Test size: 5554


In [8]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

## Bert , Fasttext , TFIDF , BOW

In [9]:
!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz

from gensim.models import KeyedVectors

fasttext_model = KeyedVectors.load_word2vec_format("cc.ar.300.vec.gz", binary=False)

--2025-07-02 05:41:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.167.112.129, 3.167.112.51, 3.167.112.66, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.167.112.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272365870 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ar.300.vec.gz’

cc.ar.300.vec.gz    100%[===================>]   1.18G  96.2MB/s    in 11s     

2025-07-02 05:41:55 (114 MB/s) - ‘cc.ar.300.vec.gz’ saved [1272365870/1272365870]



In [28]:
def evaluate_ml_model(vectorizer, model):
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    model.fit(X_train_vec, y_train_enc)
    preds = model.predict(X_test_vec)
    return accuracy_score(y_test_enc, preds)

tfidf = TfidfVectorizer(max_features=1000)
bow = CountVectorizer(max_features=1000)
log_reg = LogisticRegression()
rf = RandomForestClassifier(n_estimators=10)

results = []
for name, vec in [("TF-IDF", tfidf), ("BOW", bow)]:
    for model_name, model in [("Logistic Regression", log_reg), ("Random Forest", rf)]:
        acc = evaluate_ml_model(vec, model)
        results.append((name, model_name, acc))

In [29]:
def fasttext_embed(texts):
    embeddings = []
    for text in texts:
        vecs = [fasttext_model[word] for word in text.split() if word in fasttext_model]
        if vecs:
            embeddings.append(np.mean(vecs, axis=0))
        else:
            embeddings.append(np.zeros(300))
    return np.array(embeddings)

X_train_ft = fasttext_embed(X_train)
X_test_ft = fasttext_embed(X_test)


for model_name, model in [("Logistic Regression", log_reg), ("Random Forest", rf)]:
    model.fit(X_train_ft, y_train_enc)
    preds = model.predict(X_test_ft)
    acc = accuracy_score(y_test_enc, preds)
    results.append(("FastText", model_name, acc))


In [30]:
bert_model_name = "aubmindlab/bert-base-arabertv2"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = TFAutoModel.from_pretrained(bert_model_name)

def get_bert_cls_embeddings(texts, tokenizer, model, batch_size=16, max_len=128):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encodings = tokenizer(batch, truncation=True, padding=True, return_tensors="tf", max_length=max_len)
        outputs = model(encodings["input_ids"], attention_mask=encodings["attention_mask"])
        cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        all_embeddings.append(cls_embeddings)
        print(f"Processed batch {i//batch_size + 1} / {len(texts)//batch_size + 1}")
    return np.vstack(all_embeddings)

X_train_bert = get_bert_cls_embeddings(X_train.tolist()[:200], bert_tokenizer, bert_model)
X_test_bert = get_bert_cls_embeddings(X_test.tolist()[:50], bert_tokenizer, bert_model)

y_train_enc_small = y_train_enc[:200]
y_test_enc_small = y_test_enc[:50]


for model_name, model in [("Logistic Regression", log_reg), ("Random Forest", rf)]:
    model.fit(X_train_bert, y_train_enc_small)
    preds = model.predict(X_test_bert)
    acc = accuracy_score(y_test_enc_small, preds)
    results.append(("BERT", model_name, acc))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already u

Processed batch 1 / 13
Processed batch 2 / 13
Processed batch 3 / 13
Processed batch 4 / 13
Processed batch 5 / 13
Processed batch 6 / 13
Processed batch 7 / 13
Processed batch 8 / 13
Processed batch 9 / 13
Processed batch 10 / 13
Processed batch 11 / 13
Processed batch 12 / 13
Processed batch 13 / 13
Processed batch 1 / 4
Processed batch 2 / 4
Processed batch 3 / 4
Processed batch 4 / 4


In [31]:
def run_dl_model(X_train_array, X_test_array, y_train, y_test, mode="LSTM"):
    model = Sequential()
    model.add(Reshape((1, X_train_array.shape[1]), input_shape=(X_train_array.shape[1],)))
    if mode == "LSTM":
        model.add(LSTM(64))
    elif mode == "GRU":
        model.add(GRU(64))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train_array, y_train, epochs=1, batch_size=16, verbose=0)
    loss, acc = model.evaluate(X_test_array, y_test, verbose=0)
    return acc

tfidf = TfidfVectorizer(max_features=500)
bow = CountVectorizer(max_features=500)

X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

def fasttext_embed(texts):
    embeddings = []
    for text in texts:
        vecs = [fasttext_model[word] for word in text.split() if word in fasttext_model]
        if vecs:
            embeddings.append(np.mean(vecs, axis=0))
        else:
            embeddings.append(np.zeros(300))
    return np.array(embeddings)

X_train_ft = fasttext_embed(X_train)
X_test_ft = fasttext_embed(X_test)

for emb_name, X_tr, X_te in [
    ("TF-IDF", X_train_tfidf, X_test_tfidf),
    ("BOW", X_train_bow, X_test_bow),
    ("FastText", X_train_ft, X_test_ft)
]:
    for mode in ["LSTM", "GRU"]:
        acc = run_dl_model(X_tr, X_te, y_train_enc, y_test_enc, mode=mode)
        results.append((emb_name, mode, acc))

for mode in ["LSTM", "GRU"]:
    acc = run_dl_model(X_train_bert, X_test_bert, y_train_enc_small, y_test_enc_small, mode=mode)
    results.append(("BERT", mode, acc))

In [32]:
results_df = pd.DataFrame(results, columns=["Embedding", "Model", "Accuracy"])
results_df = results_df.sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
print("    Accuracy Results for Embeddings - Models\n")
print(results_df.to_string(index=False))

    Accuracy Results for Embeddings - Models

Embedding               Model  Accuracy
   TF-IDF Logistic Regression  0.877926
      BOW Logistic Regression  0.870904
      BOW                LSTM  0.860281
   TF-IDF                LSTM  0.858300
      BOW                 GRU  0.857400
   TF-IDF       Random Forest  0.853979
      BOW       Random Forest  0.852539
   TF-IDF                 GRU  0.852539
     BERT Logistic Regression  0.640000
 FastText       Random Forest  0.524847
 FastText Logistic Regression  0.523407
 FastText                 GRU  0.515124
 FastText                LSTM  0.498019
     BERT       Random Forest  0.460000
     BERT                 GRU  0.360000
     BERT                LSTM  0.340000


## Arabic Bert

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

def encode_bert_texts(texts, tokenizer, max_len=64):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=max_len, return_tensors="tf")

train_bert = encode_bert_texts(X_train, bert_tokenizer)
test_bert = encode_bert_texts(X_test, bert_tokenizer)

bert_model = TFAutoModelForSequenceClassification.from_pretrained(
    "aubmindlab/bert-base-arabertv2",
    num_labels=len(le.classes_)
)
optimizer, _ = create_optimizer(init_lr=2e-5, num_train_steps=100, num_warmup_steps=10)


bert_model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


bert_model.fit(train_bert.data, y_train_enc, epochs=2, batch_size=16)

bert_result = bert_model.evaluate(test_bert.data, y_test_enc)
print(f"\nArabic BERT Accuracy: {bert_result[1]:.4f}")

preds = bert_model.predict(test_bert.data).logits
pred_classes = np.argmax(preds, axis=1)

precision = precision_score(y_test_enc, pred_classes, average='weighted')
f1 = f1_score(y_test_enc, pred_classes, average='weighted')

print(f"Arabic BERT Precision: {precision:.4f}")
print(f"Arabic BERT F1 Score: {f1:.4f}")


print("\nClassification Report:")
print(classification_report(y_test_enc, pred_classes, target_names=le.classes_))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
1389/1389 [==============================] - 20124s 14s/step - loss: 1.4787 - accuracy: 0.4079
Epoch 2/2
 776/1389 [===============>..............] - ETA: 2:27:54 - loss: 1.4718 - accuracy: 0.4075

## Arabic GPT

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

gpt_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/gpt2-base-arabic")
gpt_model = TFAutoModelForSequenceClassification.from_pretrained(
    "aubmindlab/gpt2-base-arabic",
    num_labels=len(le.classes_)
)

gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def encode_gpt_texts(texts, tokenizer, max_len=64):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=max_len, return_tensors="tf")

train_gpt = encode_gpt_texts(X_train, gpt_tokenizer)
test_gpt = encode_gpt_texts(X_test, gpt_tokenizer)

gpt_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

gpt_model.fit(train_gpt.data, y_train_enc, epochs=2, batch_size=16)

gpt_result = gpt_model.evaluate(test_gpt.data, y_test_enc)
print(f"Arabic GPT Accuracy: {gpt_result[1]:.4f}")

preds = gpt_model.predict(test_gpt.data).logits
pred_classes = np.argmax(preds, axis=1)

precision = precision_score(y_test_enc, pred_classes, average='weighted')
f1 = f1_score(y_test_enc, pred_classes, average='weighted')

print(f"Arabic GPT Precision: {precision:.4f}")
print(f"Arabic GPT F1 Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_enc, pred_classes, target_names=le.classes_))